In [132]:
import pandas as pd
import re
import itertools
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import numpy as np

In [133]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [134]:
pd.set_option('display.max_rows', None)

In [135]:


df = pd.read_csv("Sentiment-Classification-FD-Reviews-master/dataset.csv") #membaca dataset FD Review
df.head()

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,Positive
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,Positive
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,Positive
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,Positive
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,Positive


In [136]:
df = df[["Review Produk","Label"]] #mengambil kolom review produk dan label saja

In [137]:
df.rename({'Review Produk': 'Teks'}, axis='columns', inplace=True)

In [138]:
df

,Teks,Label
0,Aku suka banget sama ini. Cuma ini cleanser ya...,Positive
1,Walaupun produknya mengklaim dapat digunakan s...,Positive
2,Thanks God for this product. Produk ini udah p...,Positive
3,Another favorite product from Cetaphil. Seneng...,Positive
4,awalnya kaget karena cleanser ini enggak berbu...,Positive
5,So far aku cocok sama cetaphil ini. Dan sudah ...,Positive
6,My HG cleanser! Dengan kulit yg oily tapi keri...,Positive
7,Sampai sekarang tetap setia menggunakan Cethap...,Positive
8,Really really love this cleanser. Kulit wajahk...,Positive
9,"Cetaphil bikin kulit aku sehat kembali, tadiny...",Positive


In [139]:
df.shape # melihat jumlah data dan label

(500, 2)

In [140]:
df['Label'] = df['Label'].replace(['Positive','negative'],[1,0])

In [141]:
df['Label'] = df['Label'].replace(['positive','Negative'],[1,0])

In [142]:
df

,Teks,Label
0,Aku suka banget sama ini. Cuma ini cleanser ya...,1
1,Walaupun produknya mengklaim dapat digunakan s...,1
2,Thanks God for this product. Produk ini udah p...,1
3,Another favorite product from Cetaphil. Seneng...,1
4,awalnya kaget karena cleanser ini enggak berbu...,1
5,So far aku cocok sama cetaphil ini. Dan sudah ...,1
6,My HG cleanser! Dengan kulit yg oily tapi keri...,1
7,Sampai sekarang tetap setia menggunakan Cethap...,1
8,Really really love this cleanser. Kulit wajahk...,1
9,"Cetaphil bikin kulit aku sehat kembali, tadiny...",1


In [143]:
df.Label.value_counts()

1    250
0    250
Name: Label, dtype: int64

In [144]:
#menghilangkan line baru (\n)
def remove_newline(text):
    return re.sub('\n', ' ',text)

In [145]:
#menghilangkan spasi berlebih
def remove_excessive_whitespace(text):
    return re.sub('  +', ' ', text)

In [146]:
#menghilangkan huruf yang berlebih pada kata
def remove_repeating_characters(text):
    return ''.join(''.join(s)[:1] for _, s in itertools.groupby(text))

In [147]:
#menghilangkan stopwords atau kata-kata yang tidak berpengaruh
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    return stopword.remove(text)

In [148]:
def preprocess_text(text):
    transformed_text = text.lower()
    transformed_text = remove_newline(transformed_text)
    transformed_text = remove_excessive_whitespace(transformed_text)
    transformed_text = remove_repeating_characters(transformed_text)
    transformed_text = remove_stopwords(transformed_text)
    return transformed_text

In [149]:
df['Teks_processed'] = df.Teks.apply(preprocess_text)

In [150]:
df=df[['Teks',"Teks_processed",'Label']]

In [151]:
df

,Teks,Teks_processed,Label
0,Aku suka banget sama ini. Cuma ini cleanser ya...,aku suka banget sama ini. cuma cleanser cocok ...,1
1,Walaupun produknya mengklaim dapat digunakan s...,walaupun produknya mengklaim digunakan makeup ...,1
2,Thanks God for this product. Produk ini udah p...,thanks god for this product. produk udah palin...,1
3,Another favorite product from Cetaphil. Seneng...,another favorite product from cetaphil. seneng...,1
4,awalnya kaget karena cleanser ini enggak berbu...,awalnya kaget cleanser engak berbusa sama seka...,1
5,So far aku cocok sama cetaphil ini. Dan sudah ...,so far aku cocok sama cetaphil ini. sudah purc...,1
6,My HG cleanser! Dengan kulit yg oily tapi keri...,my hg cleanser! kulit yg oily kering mengarah ...,1
7,Sampai sekarang tetap setia menggunakan Cethap...,sekarang tetap setia mengunakan cethapil clena...,1
8,Really really love this cleanser. Kulit wajahk...,realy realy love this cleanser. kulit wajahku ...,1
9,"Cetaphil bikin kulit aku sehat kembali, tadiny...","cetaphil bikin kulit aku sehat kembali, tadiny...",1


In [152]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [153]:
#proses ekstraksi data menjadi data numerik
vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(df.Teks_processed)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,10,2x,3x,4x,abis,absolutely,acid,acne,acnes,ad,...,yah,yakin,yang,yaudah,years,yes,yg,you,youtube,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.228044,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125285,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.169088,0.0,0.0,...,0.0,0.0,0.167108,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
X = words_df
y = df.Label

In [168]:
#pembagian dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30, random_state=0)

In [169]:
#membuat dan training model dengan menggunakan naive bayes
Bayes = NB()
import time
start = time.time() #menghitung waktu awal eksekusi
Bayes.fit(X_train, y_train)
stop = time.time() #menghitung waktu akhir eksekusi

In [170]:
print("Train time:", {stop - start},"s") #mendapatkan lama waktu eksekusi

Train time: {0.016993284225463867} s


In [171]:
#proses prediksi dengan menggunakan model naive bayes
y_true = y_test
start = time.time()
y_pred = Bayes.predict(X_test)
stop = time.time()
print("Testing time:", {stop - start},"s")
print(classification_report(y_true, y_pred))

Testing time: {0.011990547180175781} s
              precision    recall  f1-score   support

           0       0.68      0.80      0.73        65
           1       0.82      0.71      0.76        85

    accuracy                           0.75       150
   macro avg       0.75      0.75      0.75       150
weighted avg       0.76      0.75      0.75       150



In [172]:
#akurasi menggunakan pembagian data 7:3
accuracy = accuracy_score(y_true, y_pred)
accuracy*100

74.66666666666667

In [160]:
#akurasi menggunakan pembagian data 8:2
accuracy = accuracy_score(y_true, y_pred)
accuracy*100

78.0

In [161]:
y_pred

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1], dtype=int64)

In [162]:
np.array(y_test)

array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1], dtype=int64)

In [163]:
print(y_test)

90     0
254    1
283    0
445    1
461    1
15     1
316    0
489    0
159    1
153    1
241    0
250    1
390    1
289    0
171    1
329    1
468    0
355    1
154    1
37     0
205    1
366    1
240    0
108    1
45     0
438    1
21     1
367    1
96     0
233    0
428    0
118    1
124    1
191    0
374    1
492    0
311    0
451    1
353    1
238    0
322    0
46     0
403    0
221    1
76     0
1      1
213    1
325    1
418    0
102    1
363    1
170    1
343    1
144    0
132    0
12     1
327    1
173    1
224    1
342    1
78     0
276    0
387    1
425    0
301    1
196    0
10     1
469    0
271    1
75     1
142    0
65     1
340    1
484    0
175    1
362    1
264    1
100    1
491    0
295    0
300    1
235    0
475    0
219    1
330    1
326    1
421    0
157    1
348    0
54     1
220    1
402    0
379    1
200    1
179    0
372    0
56     1
440    1
60     1
208    1
Name: Label, dtype: int64


In [164]:
#440, 15, 159 data yang misklasifikasi

In [165]:
print(df.iloc[15])

Teks              Ini bagus buat kulit sensitif soalnya lembut b...
Teks_processed    bagus buat kulit sensitif soalnya lembut bange...
Label                                                             1
Name: 15, dtype: object


In [166]:
print(df['Teks_processed'].iloc[159])

cocok semua jenis kulit. which is dr oily sampe sensitive dry. kulit yang kadang oily bngt kadang kering bngt kadang merah2 ruam. harganya worth it lah ukuran tube yg besar (150ml) tp kalau mau coba2 dulu banyak yg jual share size online shop-ap.


In [167]:
print(df['Teks_processed'].iloc[440])

salah satu produk face cleanser terbaik indonesia. bikin muka bersih, bikin jerawat lumayan berkurang (tidak hilang semua jadi bruntusan). karena bermasalah komedo pori-pori, jadinya berpaling face cleanser lain. great product.
